In [1]:
from langchain_groq import ChatGroq


In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.


In [15]:
from langchain_community.document_loaders import WebBaseLoader
#performing web scraping
loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-itc/job/R-44289")
page_data = loader.load().pop().page_content
print(page_data)






















Software Engineer II, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu





In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer II, ITC',
  'experience': '2+ years of hands-on industry software development experience',
  'skills': ['front-end frameworks like React, Angular, or Vue.js',
   'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
   'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
   'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
   'Strong understanding of architectural design patterns and computer-science fundamentals',
   'Experience with implementing and integrating AI, Machine Learning and related data solutions'],
  'description': 'Become a part of the NIKE, Inc. Team. NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks

In [19]:
type(json_res)

list

In [23]:
!pip install pandas


  Using cached pandas-2.3.1-cp311-cp311-win_amd64.whl (11.3 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
job = json_res[0]  # Extract the dictionary from the list

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}]]

In [28]:
job

{'role': 'Software Engineer II, ITC',
 'experience': '2+ years of hands-on industry software development experience',
 'skills': ['front-end frameworks like React, Angular, or Vue.js',
  'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
  'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
  'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
  'Strong understanding of architectural design patterns and computer-science fundamentals',
  'Experience with implementing and integrating AI, Machine Learning and related data solutions'],
 'description': 'Become a part of the NIKE, Inc. Team. NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks achiever

In [32]:
job = json_res[0]
job['skills']

['front-end frameworks like React, Angular, or Vue.js',
 'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
 'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
 'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
 'Strong understanding of architectural design patterns and computer-science fundamentals',
 'Experience with implementing and integrating AI, Machine Learning and related data solutions']

In [33]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for NIKE, Inc.

Dear Hiring Manager,

I came across the job posting for a Software Engineer II, ITC at NIKE, Inc. and was impressed by the company's commitment to innovation and excellence. As a Business Development Executive at AtliQ, I believe our team can provide the expertise and solutions you need to drive your technology forward.

AtliQ is an AI & Software Consulting company with a proven track record of delivering tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has hands-on experience in software development, cloud architecture, modern DevOps, and AI/ML integrations, making us an ideal fit for your requirements.

Our capabilities align perfectly with your needs, including:

* Front-end development using React, Angular, or Vue.js (https://example.com/vue-portfolio)
* Cloud architecture and modern DevOps expertise, including infrastructure as code 